In [ ]:
# ===================================================================
# CELDA 1: Configuración Inicial, Imports y Funciones
# ===================================================================
import pandas as pd
import re
from hubspot import HubSpot
from hubspot.crm.properties import ApiException, PropertyUpdate, Option
from rut_chile import rut_chile as rc
import requests
import numpy as np
import unicodedata
from datetime import datetime, timedelta
import time
import psutil
import os
import concurrent.futures
import sys

# --- Archivo de Log ---
log_file = open("log.txt", "w")
log_file.write("====================================================\n")
log_file.write("== INICIO DEL SCRIPT: Nuevos Carga Superintendencia ==\n")
log_file.write("====================================================\n")

start_time = time.time()
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss

# --- Funciones de Ayuda (Limpieza y Validación) ---
def agregar_guion(valor):
    if pd.notna(valor) and isinstance(valor, str) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    return valor

def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception:
        return False

def limpiar_valor(valor):
    if not isinstance(valor, str):
        return ""
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()

log_file.write("Funciones de ayuda definidas.\n")

In [ ]:
# ===================================================================
# CELDA 2: Conexión y Extracción de Datos de HubSpot
# ===================================================================
log_file.write("\n--- Bloque 2: Extrayendo datos de HubSpot ---\n")
df_final_contactos = pd.DataFrame()
try:
    log_file.write("Inicializando cliente de HubSpot...\n")
    api_client = HubSpot(access_token=os.getenv('HUBSPOT_API_KEY'))
    log_file.write("Cliente de HubSpot inicializado.\n")

    properties = [ 'address','firstname','segundo_nombre', 'lastname','apellido_materno', 'condicion', 'createdate','crs',
                'date_of_birth','direccion_de_trabajo','edad','email','estado_pago','fecha_inscripcion_al_colegio',
    'fecha_titulo','gender','graduation_date','mobilephone','rcm','rut', 'lastmodifieddate', 'lifecyclestage',
                  'phone','hs_object_source_label','especialidades','fecha_act_super']

    log_file.write(f"Recuperando contactos con {len(properties)} propiedades...\n")
    all_contacts = api_client.crm.contacts.get_all(properties=properties)
    contacts_list = [contact.to_dict() for contact in all_contacts]
    log_file.write(f"Se recuperaron {len(contacts_list)} contactos.\n")

    df_contacts = pd.DataFrame(contacts_list)
    df_col_dict = pd.json_normalize(df_contacts['properties'])
    df_final_contactos = df_contacts.drop(columns=['properties']).join(df_col_dict)
    df_final_contactos = df_final_contactos.sort_values(by=['createdate'], ascending = True)
    log_file.write("DataFrames de HubSpot creados y procesados.\n")

except ApiException as e:
    log_file.write(f"!!! ERROR al conectar con HubSpot: {e}\n")
except Exception as e:
    log_file.write(f"!!! ERROR inesperado en el bloque 2: {e}\n")

In [ ]:
# ===================================================================
# CELDA 3: Procesamiento de Datos de HubSpot y Filtrado de RUTs
# ===================================================================
lista_ruts = []
df_ruts = pd.DataFrame()
if not df_final_contactos.empty:
    log_file.write("\n--- Bloque 3: Procesando datos de HubSpot y filtrando RUTs ---\n")
    df_final_contactos['rut_normalizado'] = (
        df_final_contactos['rut'].astype(str).str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero))
    df_final_contactos['rut_valido'] = df_final_contactos['rut_normalizado'].apply(validar_rut)
    df_final_contactos['rut_sdv'] = df_final_contactos['rut_normalizado'].str.split('-').str[0]
    
    df_ruts = df_final_contactos[df_final_contactos['rut_valido']].copy()
    df_ruts['fecha_act_super'] = pd.to_datetime(df_ruts['fecha_act_super'], errors='coerce')
    
    df_filtrado_rut = df_ruts[df_ruts['fecha_act_super'].isna()].copy()
    lista_ruts = df_filtrado_rut['rut_sdv'].unique().tolist()
    log_file.write(f"Se preparó una lista de {len(lista_ruts)} RUTs para consultar en la API de la Superintendencia.\n")
else:
    log_file.write("\n--- Bloque 3: Omitido por no haber datos de HubSpot ---\n")

In [ ]:
# ===================================================================
# CELDA 4: Consulta a la API de la Superintendencia de Salud
# ===================================================================
log_file.write("\n--- Bloque 4: Consultando API de la Superintendencia ---\n")
df_resultados = pd.DataFrame()

def consultar_api_super(rut):
    url_base = "https://apis.superdesalud.gob.cl/api/prestadores/rut/"
    apikey = os.getenv('SUPERDESALUD_API_KEY')
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json"}
    url = f"{url_base}{rut}.json/?apikey={apikey}"
    try:
        response = requests.get(url, headers=headers, timeout=30)
        if response.status_code == 200:
            data = response.json()
            if data and isinstance(data, list) and len(data) > 0:
                data[0]["rut_consultado"] = rut
                return data[0]
        return None
    except requests.RequestException as e:
        log_file.write(f"Error de conexión con RUT {rut}: {e}\n")
        return None

def obtener_datos_concurrente(ruts, max_workers=20):
    resultados = []
    if not ruts:
        return pd.DataFrame()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(consultar_api_super, rut): rut for rut in ruts}
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            if (i + 1) % 20 == 0 or (i + 1) == len(ruts):
                log_file.write(f"Progreso de consulta: {i+1}/{len(ruts)} RUTs procesados\n")
            resultado = future.result()
            if resultado:
                resultados.append(resultado)
    return pd.DataFrame(resultados)

if lista_ruts:
    df_resultados = obtener_datos_concurrente(lista_ruts, max_workers=20)
    log_file.write(f"Consulta a la Superintendencia finalizada. Se obtuvieron {len(df_resultados)} registros.\n")
else:
    log_file.write("No hay RUTs para consultar en la Superintendencia.\n")

In [ ]:
# ===================================================================
# CELDA 5: Procesamiento de Datos de la Superintendencia y Actualización de Opciones en HubSpot
# ===================================================================
if not df_resultados.empty:
    log_file.write("\n--- Bloque 5: Procesando datos y actualizando opciones en HubSpot ---\n")
    
    # CORRECCIÓN DEFINITIVA: Se verifica que las columnas 'rut' y 'dv' existan antes de usarlas.
    if 'rut' in df_resultados.columns and 'dv' in df_resultados.columns:
        df_resultados['rut_normalizado'] = df_resultados['rut'].astype(str) + '-' + df_resultados['dv'].astype(str)
        
        antecedentes_data = df_resultados[['rut_normalizado', 'antecedentes']]
        antecedentes_desnormalizados = antecedentes_data.explode('antecedentes').reset_index(drop=True)
        if 'antecedentes' in antecedentes_desnormalizados.columns and not antecedentes_desnormalizados['antecedentes'].isnull().all():
            antecedentes_desnormalizados = pd.concat(
                [antecedentes_desnormalizados.drop(['antecedentes'], axis=1), antecedentes_desnormalizados['antecedentes'].apply(pd.Series)],
                axis=1)

            # Lógica para actualizar las 3 propiedades (especialidades, universidad, nacionalidad)
            for prop_info in [{'name': 'especialidades', 'source': antecedentes_desnormalizados[antecedentes_desnormalizados['clase_antecedente'] == 'Especialidad'], 'column': 'cod_antecedente'},
                               {'name': 'universidad_titulo', 'source': antecedentes_desnormalizados[(antecedentes_desnormalizados['cod_antecedente'].isin(['Médica Cirujana', 'Médico Cirujano'])) & (antecedentes_desnormalizados['procedencia'] != 'EUNACOM')].drop_duplicates(subset='rut_normalizado', keep='first'), 'column': 'procedencia'},
                               {'name': 'nacionalidad', 'source': df_resultados, 'column': 'nacionalidad'}]:
                
                try:
                    property_name = prop_info['name']
                    unique_values = prop_info['source'][prop_info['column']].dropna().unique()
                    nuevas_opciones = [{"label": str(item).split(' -')[0], "value": limpiar_valor(str(item).split(' -')[0])} for item in sorted(unique_values) if str(item)]

                    properties_api = api_client.crm.properties
                    property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)
                    current_options = property_data.options if property_data.options else []
                    valores_existentes = {option.value for option in current_options}
                    
                    opciones_anadidas = 0
                    for option in nuevas_opciones:
                        if option['value'] not in valores_existentes:
                            current_options.append(Option(label=option['label'], value=option['value']))
                            valores_existentes.add(option['value'])
                            opciones_anadidas += 1
                    
                    if opciones_anadidas > 0:
                        log_file.write(f"Agregando {opciones_anadidas} nuevas opciones a la propiedad '{property_name}'...\n")
                        update_data = PropertyUpdate(options=current_options)
                        properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)
                        log_file.write(f"Propiedad '{property_name}' actualizada con éxito.\n")
                    else:
                        log_file.write(f"Propiedad '{property_name}' ya está al día.\n")
                except Exception as e:
                    log_file.write(f"!!! ERROR al actualizar opciones de la propiedad '{property_name}': {e}\n")
    else:
        log_file.write("!!! ADVERTENCIA: Columnas 'rut' y/o 'dv' no se encontraron. Se omite el procesamiento de datos de la Superintendencia.\n")
else:
    log_file.write("No hay datos de la Superintendencia para procesar.\n")

In [ ]:
# ===================================================================
# CELDA 6: Actualización de Datos de Contactos en HubSpot
# ===================================================================
if not df_resultados.empty and 'rut_normalizado' in df_resultados.columns and not df_ruts.empty:
    log_file.write("\n--- Bloque 6: Actualizando datos de contactos en HubSpot ---\n")
    try:
        df_para_actualizar = pd.merge(df_resultados, df_ruts, on='rut_normalizado', how='inner')
        df_para_actualizar = df_para_actualizar.loc[:, ~df_para_actualizar.columns.duplicated()]

        total_a_actualizar = len(df_para_actualizar)
        log_file.write(f"Iniciando la actualización de {total_a_actualizar} contactos...\n")

        for index, row in df_para_actualizar.iterrows():
            contact_id = row['id']
            fecha_nac_str = None
            try:
                fecha_nac_dt = pd.to_datetime(row.get('fecha_nacimiento'), format='%d/%m/%Y', errors='coerce')
                if pd.notna(fecha_nac_dt):
                    fecha_nac_str = fecha_nac_dt.strftime('%Y-%m-%d')
            except Exception:
                fecha_nac_str = None

            data_to_update = {
                'date_of_birth': fecha_nac_str,
                'gender': row.get('sexo'),
                'registro_superintendencia': row.get('nro_registro'),
                'nacionalidad': limpiar_valor(row.get('nacionalidad_x')),
                'firstname': row.get('nombres'),
                'lastname': row.get('apellido_paterno_x'),
                'apellido_materno': row.get('apellido_materno_x'),
                'fecha_act_super': datetime.today().strftime('%Y-%m-%d')
            }
            
            properties_payload = {k: v for k, v in data_to_update.items() if pd.notna(v)}
            
            url = f"https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}"
            headers = {'Authorization': f"Bearer {os.getenv('HUBSPOT_API_KEY')}", 'Content-Type': 'application/json'}
            response = requests.patch(url, json={'properties': properties_payload}, headers=headers)

            if response.status_code == 200:
                log_file.write(f"({index+1}/{total_a_actualizar}) Éxito: Contacto ID {contact_id} actualizado.\n")
            else:
                log_file.write(f"!!! ERROR al actualizar contacto {contact_id}: {response.status_code} - {response.text}\n")
    except Exception as e:
        log_file.write(f"!!! ERROR inesperado en el bloque 6: {e}\n")
else:
    log_file.write("\n--- Bloque 6: Omitido por no haber datos para actualizar ---\n")

In [ ]:
# ===================================================================
# CELDA FINAL: Cerrar el archivo de log
# ===================================================================
end_time = time.time()
mem_end = process.memory_info().rss
log_file.write("\n====================================================\n")
log_file.write(f"== FIN DEL SCRIPT (Duración: {end_time - start_time:.2f} segundos) ==\n")
log_file.write(f"== Memoria usada: {(mem_end - mem_start) / (1024 * 1024):.2f} MB ==\n")
log_file.write("====================================================\n")
log_file.close()